In [4]:
iopub_data_rate_limit = 10e100

In [5]:
#importing libraries
import pandas as pd
import random
import numpy as np
from itertools import combinations

In [38]:

import csv
with open('dati_covid.csv', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    macro_list=[]
    lista=[]
    
    #hashtag lists normalization
    for row in csv_reader:
        macro_list.append(row[2])
    macro_list.pop(0)
    for k in macro_list:
        k = k.replace("[","")
        k = k.replace("]","")
        k = k.replace("'","")
        k = k.replace("#",",")
        k = k.replace(";",",")
        k = k.replace("\\","")
        k = k.replace("$","\%")
        k = k.split(",")
        k = [x for x in k if x is not '']
        k = [x.strip().lower() for x in k]
        k = sorted(k)
        if len(k)<2:    #single hashtag list removing
            del k
        else:
            elem = k
            new=[c.strip() for c in elem if c.strip()]
            comb = list(combinations(new,2))   #all possible hashtag combinations in a list
        lista.append(comb)   #put each combination in a list

In [39]:
len(lista)

25303

In [40]:
#creating the dataframe
nodo1 = []
nodo2 = []

for i in range(0,len(lista)):
    for j in range(0,len(lista[i])):
        nodo1.append(lista[i][j][0])
        nodo2.append(lista[i][j][1])
dataframe = pd.DataFrame({'nodo1':nodo1, 'nodo2':nodo2})
print(dataframe)

                    nodo1             nodo2
0                 ambient              bass
1                 ambient         drum &amp
2                 ambient        indie rock
3                 ambient  psychedelic rock
4                 ambient             screw
...                   ...               ...
1775907             screw         texas rap
1775908             screw              trap
1775909  southern hip hop         texas rap
1775910  southern hip hop              trap
1775911         texas rap              trap

[1775912 rows x 2 columns]


In [41]:
dataframe['nodo1'].unique()

array(['ambient', 'bass', 'drum &amp', ..., 'octaneamy', 'raggajungle',
       'everythingispolitical'], dtype=object)

In [42]:
#computing weight for each pair of hashtag
data = dataframe.groupby(['nodo1','nodo2'],as_index=False).size().reset_index(name='peso').sort_values('peso',ascending=False)

In [43]:
data

,nodo1,nodo2,peso
279731,coronavirus,covid19,2041
279719,coronavirus,covid-19,1718
290005,coronavirus,podcast,1585
562753,hip hop,rap,996
290707,coronavirus,quarantine,950
...,...,...,...
324674,covid19,offest,1
324675,covid19,office,1
324676,covid19,officerinvolvedshooting,1
324677,covid19,official audio,1


In [44]:
#preparing data for network creation
data = data[['nodo1','nodo2','peso']]
data['peso'] = data['peso'].astype(float)

In [45]:
data = data.rename(columns={"nodo1": "#nodo1", "nodo2": "#nodo2","peso":"#peso"})
data

,#nodo1,#nodo2,#peso
279731,coronavirus,covid19,2041.0
279719,coronavirus,covid-19,1718.0
290005,coronavirus,podcast,1585.0
562753,hip hop,rap,996.0
290707,coronavirus,quarantine,950.0
...,...,...,...
324674,covid19,offest,1.0
324675,covid19,office,1.0
324676,covid19,officerinvolvedshooting,1.0
324677,covid19,official audio,1.0


In [46]:
data['#peso'].isnull().sum()

0

In [15]:
data.to_csv('final.csv',index=False) #final csv creation